In [8]:
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import openpyxl as xl
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [13]:
wb_obj = xl.load_workbook("bank_data.xlsx")
sheet_obj = wb_obj.active

tot_row = sheet_obj.max_row #the total number of rows, including the first line
tot_col = sheet_obj.max_column #the total number of col

documents = []
options = []
for i in range(2, tot_row): 
    cell_obj_statement = sheet_obj.cell(row = i, column = 2) 
    cell_obj_intent = sheet_obj.cell(row = i, column = 3)
    documents.append(cell_obj_statement.value.lower())
    options.append(cell_obj_intent.value)

vectorizer = TfidfVectorizer(max_df = 0.85) # by setting max_df to an integer, a word which has a df larger than 0.85 will be treated as a stop-word

from sklearn.model_selection import train_test_split

transformer = TfidfTransformer(smooth_idf=True)

X_tf = vectorizer.fit_transform(documents).toarray()
y = np.array(options)

In [14]:
import pickle

classifier_f = open("bank_data_classifier.pickle","rb")
our_classifier = pickle.load(classifier_f)
classifier_f.close()

In [17]:
def pred_sent(text,clf):
    word_l = word_tokenize(text.lower())
    tot_features = len(vectorizer.get_feature_names())
    text_arr = np.zeros(tot_features)
    inx = 0
    
    for w in vectorizer.get_feature_names():
        counts = 0
        for w2 in word_l:
            if w == w2:
                counts = counts +1
        text_arr[inx] = counts
        inx = inx +1
        
    x_arr = np.array([text_arr])
    x_pred = transformer.fit_transform(x_arr).toarray()
    return clf.predict(x_pred)

In [18]:
sent = input("Hangi işlemi gerçekleştirmek istiyorsunuz ?: ")

print(sent,'\n','Gerçekleştirmek istediğiniz işlem:',pred_sent(sent,our_classifier))

Hangi işlemi gerçekleştirmek istiyorsunuz ?: Havale yapmak istiyorum
Havale yapmak istiyorum 
 Gerçekleştirmek istediğiniz işlem: ['money_transfer']
